In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import ipywidgets as widgets
from IPython.display import display, Markdown

class VariableAleatoria:
    """
    Clase para manejar variables aleatorias y generar visualizaciones interactivas.
    """

    def __init__(self):
        """
        Inicializa la clase VariableAleatoria.
        """
        self.variables = {}
        self.error_output = widgets.Output()

    def agregar_variable(self, nombre, func_muestra, func_densidad, func_distribucion, parametros,
                         descripcion, interpretacion_parametros, tipo='continua', valores_default=None,
                         validar_parametros=None, tipos_parametros=None):
        """
        Agrega una nueva variable aleatoria con sus funciones asociadas y descripciones.

        Args:
            nombre (str): Nombre de la variable aleatoria.
            func_muestra (callable): Función para generar muestras de la distribución.
            func_densidad (callable): Función de densidad o masa de probabilidad.
            func_distribucion (callable): Función de distribución acumulada.
            parametros (list): Lista de nombres de parámetros de la distribución.
            descripcion (str): Descripción en Markdown de la distribución.
            interpretacion_parametros (dict): Interpretación de cada parámetro.
            tipo (str, opcional): 'continua' o 'discreta'. Por defecto 'continua'.
            valores_default (list, opcional): Valores por defecto de los parámetros.
            validar_parametros (callable, opcional): Función para validar los parámetros.
            tipos_parametros (list, opcional): Lista de tipos de los parámetros ('int' o 'float').
        """
        self.variables[nombre] = {
            'func_muestra': func_muestra,
            'func_densidad': func_densidad,
            'func_distribucion': func_distribucion,
            'parametros': parametros,
            'descripcion': descripcion,
            'interpretacion_parametros': interpretacion_parametros,
            'tipo': tipo,
            'valores_default': valores_default if valores_default else [1] * len(parametros),
            'validar_parametros': validar_parametros,
            'tipos_parametros': tipos_parametros if tipos_parametros else ['float'] * len(parametros)
        }

    def validar_parametros(self, nombre, params):
        """
        Valida los parámetros de una distribución.

        Args:
            nombre (str): Nombre de la variable aleatoria.
            params (list): Lista de valores de parámetros.

        Returns:
            tuple: (bool, str) Indica si es válido y mensaje de error en caso contrario.
        """
        validar = self.variables[nombre]['validar_parametros']
        if validar:
            try:
                validar(params)
                return True, ""
            except ValueError as e:
                return False, str(e)
        return True, ""

    def grafica_escalon_discontinuo(self, ax, limites, valores, color='b', label=None):
        """
        Grafica una función escalonada discontinua.

        Args:
            ax (Axes): Objeto Axes de Matplotlib.
            limites (array): Límites de los escalones.
            valores (array): Valores de la función en cada tramo.
            color (str, opcional): Color de la gráfica.
            label (str, opcional): Etiqueta para la leyenda.
        """
        for i in range(len(valores)):
            if i == 0 and label is not None:
                ax.hlines(valores[i], limites[i], limites[i + 1], colors=color, label=label)
            else:
                ax.hlines(valores[i], limites[i], limites[i + 1], colors=color)
            ax.plot(limites[i], valores[i], 'o', color=color)
            if i != len(valores) - 1:
              ax.plot(limites[i+1], valores[i], 'wo', markeredgecolor=color)
        ax.axhline(0, color='black', linewidth=0.5)
        ax.axvline(0, color='black', linewidth=0.5)
        ax.grid(True)

    def graficar(self, nombre, params, size, mostrar_densidad, mostrar_acumulada,
                 mostrar_masa, color_hist, color_dens, color_acum, semilla, mostrar_descripcion):
        """
        Genera las gráficas de histograma, densidad y distribución acumulada.

        Args:
            nombre (str): Nombre de la variable aleatoria.
            params (list): Lista de valores de parámetros.
            size (int): Tamaño de la muestra.
            mostrar_densidad (bool): Si se muestra la función de densidad.
            mostrar_acumulada (bool): Si se muestra la función de distribución acumulada.
            mostrar_masa (bool): Si se muestra la función de masa de probabilidad.
            color_hist (str): Color del histograma.
            color_dens (str): Color de la función de densidad o masa.
            color_acum (str): Color de la función de distribución acumulada.
            semilla (int): Semilla para la generación aleatoria.
            mostrar_descripcion (bool): Si se muestra la descripción de la distribución.
        """
        # Validar los parámetros
        valido, error_msg = self.validar_parametros(nombre, params)
        if not valido:
            with self.error_output:
                self.error_output.clear_output()
                print(f"Error: {error_msg}")
            return

        # Generar muestra
        np.random.seed(semilla)
        func_muestra = self.variables[nombre]['func_muestra']
        muestra = func_muestra(params, size)

        # Preparar texto de parámetros para títulos
        parametros = self.variables[nombre]['parametros']
        params_text = ', '.join([f"{param_name}={param_value}" for param_name, param_value in zip(parametros, params)])

        # Definir x para densidad y distribución
        tipo = self.variables[nombre]['tipo']
        if tipo == 'discreta':
            x = np.arange(min(muestra), max(muestra) + 1)
        else:
            x_min, x_max = min(muestra), max(muestra)
            x = np.linspace(x_min, x_max, 1000)

        # Crear figura con dos subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

        # Primer subplot: Histograma de la muestra
        if tipo == 'discreta':
            valores, conteos = np.unique(muestra, return_counts=True)
            frecuencias_relativas = conteos / size
            ax1.bar(valores, frecuencias_relativas, alpha=0.6, color=color_hist, label="Datos")
            max_hist = max(frecuencias_relativas)
        else:
            counts, bins, patches = ax1.hist(muestra, bins=30, density=True, alpha=0.6,
                                             color=color_hist, label="Datos")
            max_hist = max(counts)

        ax1.set_title(f"Histograma de {nombre}\n ({params_text})")
        ax1.set_xlabel("Valor")
        ax1.set_ylabel("Frecuencia relativa")

        # Función de densidad o masa de probabilidad
        mostrar_func = (mostrar_densidad and tipo == 'continua') or (mostrar_masa and tipo == 'discreta')
        if mostrar_func:
            func_densidad = self.variables[nombre]['func_densidad']
            densidad = func_densidad(params, x)
            if tipo == 'discreta':
                ax1.scatter(x, densidad, color=color_dens, label="Función de masa de probabilidad")
            else:
                ax1.plot(x, densidad, color=color_dens, lw=2, label="Función de densidad")
            # Ajustar límites del eje y
            ax1.set_ylim(0, max(max_hist, max(densidad)) * 1.1)
            # Posicionar la leyenda debajo de la gráfica
            ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=1)
        else:
            # Ajustar límite del eje y solo con el histograma
            ax1.set_ylim(0, max_hist * 1.1)

        # Segundo subplot: Histograma acumulado
        if tipo == 'discreta':
            ax2.hist(muestra, bins=np.arange(min(muestra) - 0.5, max(muestra) + 1.5),
                     density=True, cumulative=True, alpha=0.6, color=color_hist, label="Probabilidad acumulada")
        else:
            counts_cum, bins_cum, patches_cum = ax2.hist(muestra, bins=30, density=True, cumulative=True, alpha=0.6,
                                                         color=color_hist, label="Probabilidad acumulada")

        ax2.set_title(f"Probabilidad acumulada de {nombre}\n ({params_text})")
        ax2.set_xlabel("Valor")
        ax2.set_ylabel("Probabilidad acumulada")

        # Función de distribución acumulada
        if mostrar_acumulada:
            func_distribucion = self.variables[nombre]['func_distribucion']
            distribucion = func_distribucion(params, x)
            if tipo == 'discreta':
                # Graficar función escalonada con etiqueta
                limites = np.concatenate(([x[0]], x+1))
                self.grafica_escalon_discontinuo(ax2, limites, distribucion, color=color_acum, label="Función de distribución acumulada")
            else:
                ax2.plot(x, distribucion, color=color_acum, lw=2, label="Función de distribución acumulada")
            # Posicionar la leyenda debajo de la gráfica
            ax2.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=1)

        plt.tight_layout()

        # Mostrar todo
        if mostrar_descripcion:
            from ipywidgets import HBox, Output, Layout

            graphs_output = Output()
            description_output = Output()

            with graphs_output:
                plt.show()

            # Construir la descripción
            descripcion = self.variables[nombre]['descripcion']
            interpretacion_parametros = self.variables[nombre]['interpretacion_parametros']
            parametros = self.variables[nombre]['parametros']

            texto_descripcion = descripcion + "\n\n"
            for param in parametros:
                 texto_descripcion += "" #f"$\{param}$: {interpretacion_parametros[param]}\n"

            texto_descripcion = f"# Descripción de la Variable Aleatoria {nombre}\n\n" + texto_descripcion

            with description_output:
                display(Markdown(texto_descripcion))

            # Establecer layouts
            graphs_output.layout = Layout(width='65%')
            description_output.layout = Layout(width='35%')

            # Mostrar lado a lado
            display(HBox([graphs_output, description_output]))
        else:
            plt.show()

    def mostrar_interactividad(self):
        """
        Muestra la interfaz interactiva con widgets para el usuario.
        """
        # Widgets de entrada
        size_input = widgets.IntText(value=1000, description='Tamaño de muestra:',
                                     layout=widgets.Layout(width='300px'),
                                     style={'description_width': '150px'})
        semilla_input = widgets.IntText(value=0, description='Semilla:',
                                        layout=widgets.Layout(width='200px'),
                                        style={'description_width': '120px'})
        tipo_dropdown = widgets.Dropdown(options=sorted([(key, key) for key in self.variables.keys()]),
                                         description='Variable aleatoria:',
                                         layout=widgets.Layout(width='300px'),
                                         style={'description_width': '150px'})

        # Checkboxes
        densidad_checkbox = widgets.Checkbox(value=False, description="Mostrar función de densidad",
                                             layout=widgets.Layout(width='300px'),
                                             style={'description_width': '0px'})
        masa_checkbox = widgets.Checkbox(value=False, description="Mostrar función de masa de probabilidad",
                                         layout=widgets.Layout(width='300px'),
                                         style={'description_width': '0px'})
        acumulada_checkbox = widgets.Checkbox(value=False, description="Mostrar función de distribución acumulada",
                                              layout=widgets.Layout(width='300px'),
                                              style={'description_width': '0px'})
        descripcion_checkbox = widgets.Checkbox(value=True, description="Mostrar descripción",
                                                layout=widgets.Layout(width='300px'),
                                                style={'description_width': '0px'})

        # Color pickers
        color_hist = widgets.ColorPicker(value='pink', description='Color Histograma',
                                         layout=widgets.Layout(width='250px'),
                                         style={'description_width': '120px'})
        color_dens = widgets.ColorPicker(value='red', description='Color Densidad/Masa',
                                         layout=widgets.Layout(width='250px'),
                                         style={'description_width': '160px'})
        color_acum = widgets.ColorPicker(value='green', description='Color Acumulada',
                                         layout=widgets.Layout(width='250px'),
                                         style={'description_width': '120px'})

        # Crear widgets de parámetros para todos los posibles parámetros
        all_param_names = set()
        for var in self.variables.values():
            all_param_names.update(var['parametros'])

        # Crear diccionarios para almacenar los widgets y sus tipos
        parameter_widgets = {}
        params_dict = {
            'nombre': tipo_dropdown,
            'size': size_input,
            'mostrar_densidad': densidad_checkbox,
            'mostrar_acumulada': acumulada_checkbox,
            'mostrar_masa': masa_checkbox,
            'mostrar_descripcion': descripcion_checkbox,
            'color_hist': color_hist,
            'color_dens': color_dens,
            'color_acum': color_acum,
            'semilla': semilla_input,
        }

        # Crear widgets de parámetros
        for param_name in all_param_names:
            # Encontrar el tipo y valor por defecto del parámetro
            for var in self.variables.values():
                if param_name in var['parametros']:
                    index = var['parametros'].index(param_name)
                    param_tipo = var['tipos_parametros'][index]
                    default_value = var['valores_default'][index]
                    break
            # Crear el widget según el tipo
            if param_tipo == 'int':
                param_widget = widgets.IntText(value=default_value, description=param_name,
                                               layout=widgets.Layout(width='300px'),
                                               style={'description_width': '150px'})
            else:
                param_widget = widgets.FloatText(value=default_value, description=param_name,
                                                 layout=widgets.Layout(width='300px'),
                                                 style={'description_width': '150px'})
            # Añadir al diccionario de widgets
            parameter_widgets[param_name] = param_widget
            # Añadir al diccionario de parámetros para interactive_output
            params_dict[param_name] = param_widget

        # Contenedor para los widgets de parámetros
        param_widgets_box = widgets.VBox(list(parameter_widgets.values()))

        # Función para actualizar los widgets y params_dict según la distribución seleccionada
        def actualizar_widgets(change):
            nombre = tipo_dropdown.value
            parametros = self.variables[nombre]['parametros']
            valores_default = self.variables[nombre]['valores_default']
            tipos_parametros = self.variables[nombre]['tipos_parametros']
            tipo = self.variables[nombre]['tipo']

            # Mostrar u ocultar widgets de parámetros
            for param_name, param_widget in parameter_widgets.items():
                if param_name in parametros:
                    param_widget.layout.display = 'block'
                    index = parametros.index(param_name)
                    param_widget.value = valores_default[index]
                else:
                    param_widget.layout.display = 'none'

            # Mostrar u ocultar checkboxes
            if tipo == 'discreta':
                masa_checkbox.layout.display = 'block'
                densidad_checkbox.layout.display = 'none'
            else:
                masa_checkbox.layout.display = 'none'
                densidad_checkbox.layout.display = 'block'

        # Llamar a actualizar_widgets inicialmente
        actualizar_widgets(None)

        # Observar cambios en el tipo de distribución
        tipo_dropdown.observe(actualizar_widgets, names='value')

        # Función para graficar con los parámetros seleccionados
        def graficar_con_parametros(**kwargs):
            nombre = kwargs['nombre']
            size = kwargs['size']
            mostrar_densidad = kwargs['mostrar_densidad']
            mostrar_acumulada = kwargs['mostrar_acumulada']
            mostrar_masa = kwargs['mostrar_masa']
            mostrar_descripcion = kwargs['mostrar_descripcion']
            color_hist_value = kwargs['color_hist']
            color_dens_value = kwargs['color_dens']
            color_acum_value = kwargs['color_acum']
            semilla = kwargs['semilla']

            # Obtener los valores de los parámetros
            parametros = self.variables[nombre]['parametros']
            params = [kwargs[param_name] for param_name in parametros]

            with self.error_output:
                self.error_output.clear_output()

            self.graficar(nombre, params, size, mostrar_densidad, mostrar_acumulada,
                          mostrar_masa, color_hist_value, color_dens_value, color_acum_value,
                          semilla, mostrar_descripcion)

        # Crear el output interactivo
        output = widgets.interactive_output(graficar_con_parametros, params_dict)

        # Organizar widgets en columnas
        columna1 = widgets.VBox([tipo_dropdown, param_widgets_box],
                                layout=widgets.Layout(width='25%', align_items='flex-start'))
        columna2 = widgets.VBox([size_input, semilla_input],
                                layout=widgets.Layout(width='25%', align_items='flex-start'))
        columna3 = widgets.VBox([densidad_checkbox, masa_checkbox, acumulada_checkbox, descripcion_checkbox],
                                layout=widgets.Layout(width='25%', align_items='flex-start'))
        columna4 = widgets.VBox([color_hist, color_dens, color_acum],
                                layout=widgets.Layout(width='25%', align_items='flex-start'))

        hbox = widgets.HBox([columna1, columna2, columna3, columna4], layout=widgets.Layout(width='100%'))

        # Mostrar la interfaz
        display(hbox, self.error_output, output)
# Crear una instancia de la clase VariableAleatoria
va = VariableAleatoria()


# =================== Agregar Distribución Hipergeométrica =================== #

def muestra_hipergeometrica(params, size):
    N, K, n = params
    return np.random.hypergeometric(K, N - K, n, size)

def densidad_hipergeometrica(params, x):
    N, K, n = params
    return stats.hypergeom.pmf(x, N, K, n)

def distribucion_hipergeometrica(params, x):
    N, K, n = params
    return stats.hypergeom.cdf(x, N, K, n)

def validar_hipergeometrica(params):
    N, K, n = params
    if not all(isinstance(param, int) for param in params):
        raise ValueError("Los parámetros 'N', 'K' y 'n' deben ser enteros.")
    if N <= 0:
        raise ValueError("El parámetro 'N' debe ser un entero positivo.")
    if not (0 <= K <= N):
        raise ValueError("El parámetro 'K' debe estar en el intervalo [0, N].")
    if not (0 <= n <= N):
        raise ValueError("El parámetro 'n' debe estar en el intervalo [0, N].")

descripcion_hipergeometrica = r"""
La variable hipergeométrica describe el número de elementos deseados obtenidos de una urna con elementos deseados y no deseados cuando los extraemos de la urna sin reemplazo. Los parámetros de esta distribución son $N$, el número total de elementos en la urna, $K$, el número de elementos deseados en la urna (por lo que hay $N - K$ elementos no deseados en la urna), y $n$, el número de elementos obtenidos de la urna.

La función de probabilidad puntual está dada por

$$
p_k = P[X = k] = \begin{cases}
\displaystyle \frac{\binom{K}{k} \binom{N - K}{n - k}}{\binom{N}{n}}, & k \in \{ \mathrm{max}(0, n + K - N), \ldots, \mathrm{min}(n, K) \} \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada está dada por

$$
F[x] = P[X \leq x] = \begin{cases}
\displaystyle \sum_{j = \mathrm{max}(0, n + K - N)}^{\lfloor x \rfloor} \frac{\binom{K}{j} \binom{N - K}{n - j}}{\binom{N}{n}}, & x \in [\mathrm{max}(0, n + K - N), \mathrm{min}(n, K)] \\
1, & x > \mathrm{min}(n, K) \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_hipergeometrica = {
    'N': "Tamaño de la población total (entero positivo).",
    'K': "Número total de éxitos en la población (0 ≤ K ≤ N).",
    'n': "Tamaño de la muestra extraída (0 ≤ n ≤ N)."
}

va.agregar_variable(
    "Hipergeométrica",
    muestra_hipergeometrica,
    densidad_hipergeometrica,
    distribucion_hipergeometrica,
    ['N', 'K', 'n'],
    descripcion_hipergeometrica,
    interpretacion_hipergeometrica,
    tipo='discreta',
    valores_default=[50, 10, 5],
    validar_parametros=validar_hipergeometrica,
    tipos_parametros=['int', 'int', 'int']  # Especificamos que los parámetros son enteros
)

# =================== Agregar Distribución Weibull =================== #

def muestra_weibull(params, size):
    k, lam = params
    return np.random.weibull(k, size) * lam

def densidad_weibull(params, x):
    k, lam = params
    return stats.weibull_min.pdf(x, k, scale=lam)

def distribucion_weibull(params, x):
    k, lam = params
    return stats.weibull_min.cdf(x, k, scale=lam)

def validar_weibull(params):
    k, lam = params
    if k <= 0 or lam <= 0:
        raise ValueError("Los parámetros 'k' y 'λ' deben ser mayores que 0.")


descripcion_weibull = r"""
La distribución Weibull es ampliamente utilizada en análisis de supervivencia y confiabilidad para modelar tiempos hasta fallo. Tiene dos parámetros: $k$ (forma) y $\lambda$ (escala).

Función de densidad:

$$
f(x) = \begin{cases}
\displaystyle \frac{k}{\lambda} \left( \frac{x}{\lambda} \right )^{ k - 1 } e^{ - \left( \frac{x}{\lambda} \right )^{ k } }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$
Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \begin{cases}
1 - e^{ - \left( \frac{x}{\lambda} \right )^{ k } }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$
"""
interpretacion_weibull = {
    'k': "Parámetro de forma (k > 0).",
    'λ': "Parámetro de escala (λ > 0)."
}

va.agregar_variable(
    "Weibull",
    muestra_weibull,
    densidad_weibull,
    distribucion_weibull,
    ['k', 'λ'],
    descripcion_weibull,
    interpretacion_weibull,
    tipo='continua',
    valores_default=[1.5, 1],
    validar_parametros=validar_weibull,
    tipos_parametros=['float', 'float']
)

# =================== Agregar Distribución Log-Normal =================== #

def muestra_log_normal(params, size):
    mu, sigma = params
    return np.random.lognormal(mean=mu, sigma=sigma, size=size)

def densidad_log_normal(params, x):
    mu, sigma = params
    return stats.lognorm.pdf(x, s=sigma, scale=np.exp(mu))

def distribucion_log_normal(params, x):
    mu, sigma = params
    return stats.lognorm.cdf(x, s=sigma, scale=np.exp(mu))

def validar_log_normal(params):
    mu, sigma = params
    if sigma <= 0:
        raise ValueError("El parámetro 'sigma' debe ser mayor que 0.")

descripcion_log_normal = r"""
Las variables log-normal modelan procesos en los cuales pequeños cambios se acumulan a través del tiempo. Por ejemplo, la longitud de un tejido, la duración de un juego de ajedrez, el tamaño de partículas de polímeros. Los parámetros de la log-normal son $\mu$ (posición) y $\sigma^2$ (dispersión).

Si $N$ es una variable aleatoria normal con parámetros $\mu$ y $\sigma$, entonces $X = e^N$ es una variable aleatoria log-normal con parámetros $\mu$ y $\sigma$.

Función de densidad:

$$
f(x) = \begin{cases}
\displaystyle \frac{1}{x \sigma \sqrt{2\pi}} e^{ -\frac{1}{2} \left( \frac{ \ln(x) - \mu }{ \sigma } \right )^2 }, & x > 0 \\
0, & \text{o.c.}
\end{cases}
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \begin{cases}
\displaystyle \int_{0}^{x} \frac{1}{u \sigma \sqrt{2\pi}} e^{ -\frac{1}{2} \left( \frac{ \ln(u) - \mu }{ \sigma } \right )^2 } \mathrm{d}u, & x > 0 \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_log_normal = {
    'mu': "Media del logaritmo natural de la variable.",
    'sigma': "Desviación estándar del logaritmo natural (σ > 0)."
}

va.agregar_variable(
    "Log-Normal",
    muestra_log_normal,
    densidad_log_normal,
    distribucion_log_normal,
    ['mu', 'sigma'],
    descripcion_log_normal,
    interpretacion_log_normal,
    tipo='continua',
    valores_default=[0, 1],
    validar_parametros=validar_log_normal,
    tipos_parametros=['float', 'float']
)

# =================== Agregar Distribución Binomial Negativa =================== #

def muestra_binomial_negativa(params, size):
    r, p = params
    return np.random.negative_binomial(r, p, size)

def densidad_binomial_negativa(params, x):
    r, p = params
    return stats.nbinom.pmf(x, r, p)

def distribucion_binomial_negativa(params, x):
    r, p = params
    return stats.nbinom.cdf(x, r, p)

def validar_binomial_negativa(params):
    r, p = params
    if not isinstance(r, int) or r <= 0:
        raise ValueError("El parámetro 'n' debe ser un entero positivo.")
    if not (0 < p <= 1):
        raise ValueError("El parámetro 'p' debe estar en el intervalo (0, 1].")
descripcion_binomial_negativa = r"""
La variable aleatoria binomial negativa cuenta cuántas veces fue necesario repetir un experimento para obtener un cierto número de resultados deseados. Los parámetros de esta distribución son: $n$, el número de resultados positivos requeridos, y $p$, la probabilidad del resultado deseado. Es la generalización de la geométrica.

Su función de probabilidad puntual es:

$$
p_k = P[X = k] = \begin{cases}
\displaystyle \binom{k - 1}{n - 1} p^n (1 - p)^{k - n}, & k \in \{ n, n + 1, \ldots \} \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es:

$$
F(x) = P[X \leq x] = \begin{cases}
\displaystyle \sum_{j = n}^{\lfloor x \rfloor} \binom{j - 1}{n - 1} p^n (1 - p)^{j - n}, & x \geq n \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_binomial_negativa = {
    'r': "Número de éxitos requeridos (entero positivo).",
    'p': "Probabilidad de éxito en cada ensayo (0 < p ≤ 1)."
}

va.agregar_variable(
    "Binomial Negativa",
    muestra_binomial_negativa,
    densidad_binomial_negativa,
    distribucion_binomial_negativa,
    ['n', 'p'],
    descripcion_binomial_negativa,
    interpretacion_binomial_negativa,
    tipo='discreta',
    valores_default=[5, 0.5],
    validar_parametros=validar_binomial_negativa,
    tipos_parametros=['int', 'float']
)
# =================== Agregar Distribución Geométrica =================== #

def muestra_geometrica(params, size):
    p = params[0]
    return np.random.geometric(p, size)

def densidad_geometrica(params, x):
    p = params[0]
    return stats.geom.pmf(x, p)

def distribucion_geometrica(params, x):
    p = params[0]
    return stats.geom.cdf(x, p)

def validar_geometrica(params):
    p = params[0]
    if not (0 < p <= 1):
        raise ValueError("El parámetro 'p' debe estar en el intervalo (0, 1].")

descripcion_geometrica = r"""
La variable aleatoria geométrica indica el número de intentos necesarios antes de obtener un resultado deseado. Su parámetro es $p$, la probabilidad de tener el resultado deseado en cada intento.

Su función de probabilidad puntual se define como

$$
p_k = P[X = k] = \begin{cases}
(1 - p)^{k - 1} p, & k \in \mathbb{N} \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es

$$
F[x] = P[X \leq x] = \begin{cases}
1 - (1 - p)^{\lfloor x \rfloor}, & x \geq 1 \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_geometrica = {
    'p': "Probabilidad de éxito en cada ensayo (0 < p ≤ 1)."
}

va.agregar_variable(
    "Geométrica",
    muestra_geometrica,
    densidad_geometrica,
    distribucion_geometrica,
    ['p'],
    descripcion_geometrica,
    interpretacion_geometrica,
    tipo='discreta',
    valores_default=[0.5],
    validar_parametros=validar_geometrica,
    tipos_parametros=['float']
)

# =================== Agregar Distribución Binomial =================== #

def muestra_binomial(params, size):
    n, p = params
    return np.random.binomial(n, p, size)

def densidad_binomial(params, x):
    n, p = params
    return stats.binom.pmf(x, n, p)

def distribucion_binomial(params, x):
    n, p = params
    return stats.binom.cdf(x, n, p)

def validar_binomial(params):
    n, p = params
    if not isinstance(n, int) or n <= 0:
        raise ValueError("El parámetro 'n' debe ser un entero positivo.")
    if not (0 <= p <= 1):
        raise ValueError("El parámetro 'p' debe estar en el intervalo [0, 1].")

descripcion_binomial = r"""
La distribución binomial modela el número de resultados deseados cuando se repite un experimento $n$ veces de manera independiente, cuando la probabilidad del resultado deseado es $p$ (y por consiguiente la probabilidad de un resultado no deseado es $1 - p$). Una variable aleatoria binomial es la suma de $n$ variables Bernoulli con parámetro $p$. Los parámetros de esta distribución son $n$ y $p$.

La función de probabilidad puntual de esta distribución es

$$
p_k = P[X = k] = \begin{cases}
\binom{n}{k} p^k (1 - p)^{n - k}, & k \in \{0, 1, \ldots, n\} \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es

$$
F[x] = P[X \leq x] = \sum_{j=0}^{\lfloor x \rfloor} \binom{n}{j} p^j (1 - p)^{n - j}
$$
"""


interpretacion_binomial = {
    'n': "Número de ensayos (entero positivo).",
    'p': "Probabilidad de éxito en cada ensayo (0 ≤ p ≤ 1)."
}

va.agregar_variable(
    "Binomial",
    muestra_binomial,
    densidad_binomial,
    distribucion_binomial,
    ['n', 'p'],
    descripcion_binomial,
    interpretacion_binomial,
    tipo='discreta',
    valores_default=[10, 0.5],
    validar_parametros=validar_binomial,
    tipos_parametros=['int', 'float']
)

# =================== Agregar Distribución Cauchy =================== #

def muestra_cauchy(params, size):
    x0, gamma = params
    return stats.cauchy.rvs(loc=x0, scale=gamma, size=size)

def densidad_cauchy(params, x):
    x0, gamma = params
    return stats.cauchy.pdf(x, loc=x0, scale=gamma)

def distribucion_cauchy(params, x):
    x0, gamma = params
    return stats.cauchy.cdf(x, loc=x0, scale=gamma)

def validar_cauchy(params):
    x0, gamma = params
    if gamma <= 0:
        raise ValueError("El parámetro 'gamma' debe ser mayor que 0.")

descripcion_cauchy = r"""
La distribución Cauchy tiene aplicaciones en Física. Sin embargo, es una distribución famosa pues es estable y patológica, ya que no tiene media ni varianza ni ningún otro momento. Tiene dos parámetros: $x_0$ (posición de la moda) y $\gamma$ (dispersión).

Función de densidad:

$$
f(x) = \frac{1}{\pi \gamma \left[ 1 + \left( \frac{ x - x_0 }{ \gamma } \right )^2 \right ] }
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \frac{1}{\pi} \arctan\left( \frac{ x - x_0 }{ \gamma } \right ) + \frac{1}{2}
$$
"""


interpretacion_cauchy = {
    'x0': "Localización del pico máximo de la distribución.",
    'gamma': "Escala que determina el ancho de la distribución (gamma > 0)."
}

va.agregar_variable(
    "Cauchy",
    muestra_cauchy,
    densidad_cauchy,
    distribucion_cauchy,
    ['x0', 'gamma'],
    descripcion_cauchy,
    interpretacion_cauchy,
    tipo='continua',
    valores_default=[0, 1],
    validar_parametros=validar_cauchy,
    tipos_parametros=['float', 'float']
)

# =================== Agregar Distribución Bernoulli =================== #

def muestra_bernoulli(params, size):
    p = params[0]
    return np.random.binomial(1, p, size)

def densidad_bernoulli(params, x):
    p = params[0]
    return stats.bernoulli.pmf(x, p)

def distribucion_bernoulli(params, x):
    p = params[0]
    return stats.bernoulli.cdf(x, p)

def validar_bernoulli(params):
    p = params[0]
    if not (0 <= p <= 1):
        raise ValueError("El parámetro 'p' debe estar en el intervalo [0, 1].")

descripcion_bernoulli = r"""
Esta variable aleatoria representa un experimento en el cual sólo hay dos resultados posibles, uno deseado y uno no deseado. El parámetro de esta distribución es $p$, que representa la probabilidad de obtener el resultado deseado (por consiguiente, el resultado no deseado tiene probabilidad $1 - p$).

Su probabilidad puntual está dada por

$$
p_k = P[X = k] = \begin{cases}
1 - p, & k = 0 \\
p, & k = 1 \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es

$$
F[x] = P[X \leq x] = \begin{cases}
1 - p, & x \in [0, 1) \\
1, & x \geq 1 \\
0, & \text{o.c.}
\end{cases}
$$
"""

interpretacion_bernoulli = {
    'p': "Probabilidad de éxito (0 ≤ p ≤ 1)."
}

va.agregar_variable(
    "Bernoulli",
    muestra_bernoulli,
    densidad_bernoulli,
    distribucion_bernoulli,
    ['p'],
    descripcion_bernoulli,
    interpretacion_bernoulli,
    tipo='discreta',
    valores_default=[0.5],
    validar_parametros=validar_bernoulli,
    tipos_parametros=['float']
)
# =================== Agregar Distribución t de Student =================== #

def muestra_t_student(params, size):
    df = params[0]
    return np.random.standard_t(df, size)

def densidad_t_student(params, x):
    df = params[0]
    return stats.t.pdf(x, df)

def distribucion_t_student(params, x):
    df = params[0]
    return stats.t.cdf(x, df)

def validar_t_student(params):
    df = params[0]
    if df <= 0:
        raise ValueError("El parámetro 'grados de libertad' debe ser mayor que 0.")

descripcion_t_student = r"""
La distribución t de Student se utiliza en pruebas de hipótesis cuando se desconoce la desviación estándar de la población y el tamaño de muestra es pequeño. Tiene un parámetro, $\nu$, que representa los grados de libertad.

Función de densidad:

$$
f(x) = \displaystyle \frac{ \Gamma\left( \frac{ \nu + 1 }{ 2 } \right ) }{ \sqrt{ \nu \pi } \, \Gamma\left( \frac{ \nu }{ 2 } \right ) } \left( 1 + \frac{ x^2 }{ \nu } \right )^{ - \frac{ \nu + 1 }{ 2 } }
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \frac{1}{2} + x \cdot \frac{ \Gamma\left( \frac{ \nu + 1 }{ 2 } \right ) }{ \sqrt{ \nu \pi } \, \Gamma\left( \frac{ \nu }{ 2 } \right ) } \cdot {}_2F_1 \left( \frac{1}{2}, \frac{ \nu + 1 }{ 2 }; \frac{3}{2}; - \frac{ x^2 }{ \nu } \right )
$$

Donde ${}_2F_1$ es la función hipergeométrica.

"""

interpretacion_t_student = {
    'grados de libertad': "Grados de libertad (ν > 0)."
}

va.agregar_variable(
    "t de Student",
    muestra_t_student,
    densidad_t_student,
    distribucion_t_student,
    ['grados de libertad'],
    descripcion_t_student,
    interpretacion_t_student,
    tipo='continua',
    valores_default=[10],
    validar_parametros=validar_t_student,
    tipos_parametros=['float']
)

# =================== Agregar Distribución Chi-cuadrado =================== #

def muestra_chi_cuadrado(params, size):
    df = params[0]
    return np.random.chisquare(df, size)

def densidad_chi_cuadrado(params, x):
    df = params[0]
    return stats.chi2.pdf(x, df)

def distribucion_chi_cuadrado(params, x):
    df = params[0]
    return stats.chi2.cdf(x, df)

def validar_chi_cuadrado(params):
    df = params[0]
    if df <= 0:
        raise ValueError("El parámetro 'grados de libertad' debe ser mayor que 0.")

descripcion_chi_cuadrado = r"""
La distribución $\chi^2$ (chi-cuadrado) se utiliza en pruebas de hipótesis y en análisis de varianza. Representa la suma de los cuadrados de $k$ variables aleatorias independientes y estándar normales. Su único parámetro es $k$, que representa los grados de libertad.

Función de densidad:

$$
f(x) = \begin{cases}
\displaystyle \frac{1}{2^{ k / 2 } \Gamma\left( \frac{ k }{ 2 } \right ) } x^{ \frac{ k }{ 2 } - 1 } e^{ - x / 2 }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \frac{ \gamma\left( \frac{ k }{ 2 }, \frac{ x }{ 2 } \right ) }{ \Gamma\left( \frac{ k }{ 2 } \right ) }
$$

Donde $\gamma$ es la función gamma incompleta inferior.
"""
interpretacion_chi_cuadrado = {
    'grados de libertad': "Grados de libertad (k > 0)."
}

va.agregar_variable(
    "Chi-cuadrado",
    muestra_chi_cuadrado,
    densidad_chi_cuadrado,
    distribucion_chi_cuadrado,
    ['grados de libertad'],
    descripcion_chi_cuadrado,
    interpretacion_chi_cuadrado,
    tipo='continua',
    valores_default=[5],
    validar_parametros=validar_chi_cuadrado,
    tipos_parametros=['float']
)

# =================== Agregar Distribución F de Fisher =================== #

def muestra_fisher(params, size):
    dfn, dfd = params
    return np.random.f(dfn, dfd, size)

def densidad_fisher(params, x):
    dfn, dfd = params
    return stats.f.pdf(x, dfn, dfd)

def distribucion_fisher(params, x):
    dfn, dfd = params
    return stats.f.cdf(x, dfn, dfd)

def validar_fisher(params):
    dfn, dfd = params
    if dfn <= 0 or dfd <= 0:
        raise ValueError("Los parámetros 'd1' y 'd2' deben ser mayores que 0.")

descripcion_fisher = r"""
La distribución F de Fisher-Snedecor se utiliza principalmente en análisis de varianza (ANOVA) y pruebas de hipótesis relacionadas con la igualdad de varianzas. Tiene dos parámetros: $d_1$ y $d_2$, que son los grados de libertad del numerador y del denominador, respectivamente.

Función de densidad:

$$
f(x) = \begin{cases}
\displaystyle \frac{ \left( \frac{ d_1 x }{ d_2 } \right )^{ d_1 / 2 } \left( 1 + \frac{ d_1 x }{ d_2 } \right )^{ - ( d_1 + d_2 ) / 2 } }{ x B\left( \frac{ d_1 }{ 2 }, \frac{ d_2 }{ 2 } \right ) }, & x > 0 \\
0, & \text{o.c.}
\end{cases}
$$

Donde $B$ es la función Beta.

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = I_{ \frac{ d_1 x }{ d_1 x + d_2 } } \left( \frac{ d_1 }{ 2 }, \frac{ d_2 }{ 2 } \right )
$$

Donde $I$ es la función beta incompleta regularizada.
"""

interpretacion_fisher = {
    'dfn': "Grados de libertad del numerador (d₁ > 0).",
    'dfd': "Grados de libertad del denominador (d₂ > 0)."
}

va.agregar_variable(
    "F de Fisher",
    muestra_fisher,
    densidad_fisher,
    distribucion_fisher,
    ['d1', 'd2'],
    descripcion_fisher,
    interpretacion_fisher,
    tipo='continua',
    valores_default=[5, 2],
    validar_parametros=validar_fisher,
    tipos_parametros=['float', 'float']
)

# =================== Agregar Distribución Gamma =================== #

def muestra_gamma(params, size):
    k, theta = params
    return np.random.gamma(k, theta, size)

def densidad_gamma(params, x):
    k, theta = params
    return stats.gamma.pdf(x, a=k, scale=theta)

def distribucion_gamma(params, x):
    k, theta = params
    return stats.gamma.cdf(x, a=k, scale=theta)

def validar_gamma(params):
    k, theta = params
    if k <= 0 or theta <= 0:
        raise ValueError("Los parámetros 'alfa' y 'beta' deben ser mayores que 0.")

descripcion_gamma = r"""
La distribución Gamma tiene parámetros $\alpha$ (forma) y $\beta$ (tasa). Cuando $\alpha \in \mathbb{N}$, la distribución Gamma representa la suma de $\alpha$ variables aleatorias exponenciales con parámetro $\beta$, por lo que la distribución Gamma generaliza a la exponencial.

Su función de densidad es:

$$
f(x) = \begin{cases}
\displaystyle \frac{ \beta^{ \alpha } }{ \Gamma( \alpha ) } x^{ \alpha - 1 } e^{ - \beta x }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es:

$$
F(x) = P[X \leq x] = \begin{cases}
\displaystyle \int_{0}^{x} \frac{ \beta^{ \alpha } }{ \Gamma( \alpha ) } u^{ \alpha - 1 } e^{ - \beta u } \mathrm{d}u, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_gamma = {
    'k': "Parámetro de forma (k > 0).",
    'θ': "Parámetro de escala (θ > 0)."
}

va.agregar_variable(
    "Gamma",
    muestra_gamma,
    densidad_gamma,
    distribucion_gamma,
    ['alfa', 'beta'],
    descripcion_gamma,
    interpretacion_gamma,
    tipo='continua',
    valores_default=[2, 2],
    validar_parametros=validar_gamma,
    tipos_parametros=['float', 'float']
)

# =================== Agregar Distribución Poisson =================== #

def muestra_poisson(params, size):
    lam = params[0]
    return np.random.poisson(lam, size)

def densidad_poisson(params, x):
    lam = params[0]
    return stats.poisson.pmf(x, lam)

def distribucion_poisson(params, x):
    lam = params[0]
    return stats.poisson.cdf(x, lam)

def validar_poisson(params):
    lam = params[0]
    if lam <= 0:
        raise ValueError("El parámetro 'lambda' debe ser mayor que 0.")

descripcion_poisson = r"""
La variable aleatoria Poisson cuenta cuántas veces sucede un evento específico en un periodo de tiempo fijo. La distribución tiene parámetro $\lambda$, que representa el número promedio de eventos que ocurren en el periodo de tiempo.

Su función de probabilidad puntual es:

$$
p_k = P[X = k] = \begin{cases}
\displaystyle \frac{ \lambda^{k} e^{ -\lambda } }{ k! }, & k \in \mathbb{N}_0 \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es:

$$
F(x) = P[X \leq x] = \begin{cases}
\displaystyle \sum_{j = 0}^{ \lfloor x \rfloor } \frac{ \lambda^{j} e^{ -\lambda } }{ j! }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_poisson = {
    'lambda': "Tasa media de ocurrencia de eventos (λ > 0)."
}

va.agregar_variable(
    "Poisson",
    muestra_poisson,
    densidad_poisson,
    distribucion_poisson,
    ['lambda'],
    descripcion_poisson,
    interpretacion_poisson,
    tipo='discreta',
    valores_default=[5],
    validar_parametros=validar_poisson,
    tipos_parametros=['float']
)

# =================== Agregar Distribución Uniforme Discreta =================== #

def muestra_uniforme_discreta(params, size):
    a, b = params
    return np.random.randint(a, b + 1, size)

def densidad_uniforme_discreta(params, x):
    a, b = params
    probs = np.ones_like(x) / (b - a + 1)
    probs[(x < a) | (x > b)] = 0
    return probs

def distribucion_uniforme_discreta(params, x):
    a, b = params
    cdf = (np.floor(x) - a + 1) / (b - a + 1)
    cdf[x < a - 0.5] = 0
    cdf[x > b + 0.5] = 1
    return cdf

def validar_uniforme_discreta(params):
    a, b = params
    if not all(isinstance(param, int) for param in params):
        raise ValueError("Los parámetros 'a' y 'b' deben ser enteros.")
    if a >= b:
        raise ValueError("El parámetro 'a' debe ser menor que 'b'.")

descripcion_uniforme_discreta = r"""
# Distribución Uniforme Discreta

Es la distribución de $n < \infty$ elementos cuando todos son igualmente probables. El parámetro de esta distribución es $n$ (número de resultados equiprobables).

Su función de probabilidad puntual es

$$
p_k = P[X = k] = \begin{cases}
\frac{1}{n}, & k \in \{1, \ldots, n\} \\
0, & \text{o.c.}
\end{cases}
$$

Su función de distribución acumulada es

$$
F[x] = P[X \leq x] = \begin{cases}
\frac{\lfloor x \rfloor}{n}, & x \in [1, n] \\
1, & x > n \\
0, & \text{o.c.}
\end{cases}
$$
"""

interpretacion_uniforme_discreta = {
    'a': "Límite inferior entero (a < b).",
    'b': "Límite superior entero."
}

va.agregar_variable(
    "Uniforme Discreta",
    muestra_uniforme_discreta,
    densidad_uniforme_discreta,
    distribucion_uniforme_discreta,
    ['a', 'b'],
    descripcion_uniforme_discreta,
    interpretacion_uniforme_discreta,
    tipo='discreta',
    valores_default=[1, 6],
    validar_parametros=validar_uniforme_discreta,
    tipos_parametros=['int', 'int']
)

# =================== Agregar Distribución Uniforme Continua =================== #

def muestra_uniforme_continua(params, size):
    a, b = params
    return np.random.uniform(a, b, size)

def densidad_uniforme_continua(params, x):
    a, b = params
    return stats.uniform.pdf(x, loc=a, scale=b - a)

def distribucion_uniforme_continua(params, x):
    a, b = params
    return stats.uniform.cdf(x, loc=a, scale=b - a)

def validar_uniforme_continua(params):
    a, b = params
    if a >= b:
        raise ValueError("El parámetro 'a' debe ser menor que 'b'.")

descripcion_uniforme_continua = r"""
La variable aleatoria continua uniforme representa un número infinito y continuo de posibilidades equiprobables. El parámetro de esta distribución es el intervalo $[a, b]$ en el cual los resultados son equiprobables.

Función de densidad:

$$
f(x) = \begin{cases}
\displaystyle \frac{1}{b - a}, & x \in [a, b] \\
0, & \text{o.c.}
\end{cases}
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \begin{cases}
\displaystyle \frac{ x - a }{ b - a }, & x \in [a, b] \\
1, & x > b \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_uniforme_continua = {
    'a': "Límite inferior del intervalo (a < b).",
    'b': "Límite superior del intervalo."
}

va.agregar_variable(
    "Uniforme Continua",
    muestra_uniforme_continua,
    densidad_uniforme_continua,
    distribucion_uniforme_continua,
    ['a', 'b'],
    descripcion_uniforme_continua,
    interpretacion_uniforme_continua,
    tipo='continua',
    valores_default=[0, 1],
    validar_parametros=validar_uniforme_continua,
    tipos_parametros=['float', 'float']
)


# =================== Agregar Distribución Beta =================== #

def muestra_beta(params, size):
    a, b = params
    return np.random.beta(a, b, size)

def densidad_beta(params, x):
    a, b = params
    return stats.beta.pdf(x, a, b)

def distribucion_beta(params, x):
    a, b = params
    return stats.beta.cdf(x, a, b)

def validar_beta(params):
    a, b = params
    if a <= 0 or b <= 0:
        raise ValueError("Los parámetros 'alfa' y 'beta' deben ser mayores que 0.")

descripcion_beta = r"""
La distribución Beta generaliza a la uniforme: sus variables aleatorias pueden tener cualquier valor en el intervalo $[0, 1]$ (o en cualquier intervalo, tras una transformación). Como está definida en este intervalo, entonces la distribución Beta es la distribución de probabilidades aleatorias. Sus parámetros son $\alpha$ y $\beta$, las cuales indican la forma de la densidad.

Función de densidad:

$$
f(x) = \begin{cases}
\displaystyle \frac{ x^{ \alpha - 1 } (1 - x)^{ \beta - 1 } }{ B( \alpha, \beta ) }, & x \in [0, 1] \\
0, & \text{o.c.}
\end{cases}
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \begin{cases}
\displaystyle \int_{0}^{x} \frac{ u^{ \alpha - 1 } (1 - u)^{ \beta - 1 } }{ B( \alpha, \beta ) } \mathrm{d}u, & x \in [0, 1] \\
1, & x > 1 \\
0, & \text{o.c.}
\end{cases}
$$

Donde $B( \alpha, \beta )$ es la función Beta.
"""


interpretacion_beta = {
    'a': "Parámetro de forma a (a > 0).",
    'b': "Parámetro de forma b (b > 0)."
}

va.agregar_variable(
    "Beta",
    muestra_beta,
    densidad_beta,
    distribucion_beta,
    ['alfa', 'beta'],
    descripcion_beta,
    interpretacion_beta,
    tipo='continua',
    valores_default=[2, 2],
    validar_parametros=validar_beta,
    tipos_parametros=['float', 'float']
)


# =================== Agregar Distribución Exponencial =================== #

def muestra_exponencial(params, size):
    lam = params[0]
    return np.random.exponential(1 / lam, size)

def densidad_exponencial(params, x):
    lam = params[0]
    return stats.expon.pdf(x, scale=1 / lam)

def distribucion_exponencial(params, x):
    lam = params[0]
    return stats.expon.cdf(x, scale=1 / lam)

def validar_exponencial(params):
    lam = params[0]
    if lam <= 0:
        raise ValueError("El parámetro 'lambda' debe ser mayor que 0.")

descripcion_exponencial = r"""
Una variable exponencial describe el tiempo de espera hasta que cierto evento suceda. Esta distribución tiene un parámetro, $\lambda$, el cual es el recíproco del tiempo promedio que se debe esperar a que pase el evento.

Función de densidad:

$$
f(x) = \begin{cases}
\lambda e^{ -\lambda x }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \begin{cases}
1 - e^{ -\lambda x }, & x \geq 0 \\
0, & \text{o.c.}
\end{cases}
$$
"""


interpretacion_exponencial = {
    'lambda': "Tasa de ocurrencia de eventos (λ > 0)."
}

va.agregar_variable(
    "Exponencial",
    muestra_exponencial,
    densidad_exponencial,
    distribucion_exponencial,
    ['lambda'],
    descripcion_exponencial,
    interpretacion_exponencial,
    tipo='continua',
    valores_default=[1],
    validar_parametros=validar_exponencial,
    tipos_parametros=['float']
)

# =================== Agregar Distribución Normal =================== #

def muestra_normal(params, size):
    mu, sigma = params
    return np.random.normal(mu, sigma, size)

def densidad_normal(params, x):
    mu, sigma = params
    return stats.norm.pdf(x, loc=mu, scale=sigma)

def distribucion_normal(params, x):
    mu, sigma = params
    return stats.norm.cdf(x, loc=mu, scale=sigma)

def validar_normal(params):
    mu, sigma = params
    if sigma <= 0:
        raise ValueError("El parámetro 'sigma' debe ser mayor que 0.")

descripcion_normal = r"""
La distribución normal es la distribución de datos experimentales, es decir, de datos aleatorios sumados y promediados. Tiene dos parámetros $\mu$ (media) y $\sigma^2$ (varianza).

Función de densidad:

$$
f(x) = \frac{1}{ \sigma \sqrt{ 2\pi } } e^{ -\frac{1}{2} \left( \frac{ x - \mu }{ \sigma } \right )^2 }
$$

Función de distribución acumulada:

$$
F(x) = P[X \leq x] = \int_{ -\infty }^{ x } \frac{1}{ \sigma \sqrt{ 2\pi } } e^{ -\frac{1}{2} \left( \frac{ u - \mu }{ \sigma } \right )^2 } \mathrm{d}u
$$
"""


interpretacion_normal = {
    'mu': "Media de la distribución.",
    'sigma': "Desviación estándar (σ > 0)."
}

va.agregar_variable(
    "Normal",
    muestra_normal,
    densidad_normal,
    distribucion_normal,
    ['mu', 'sigma'],
    descripcion_normal,
    interpretacion_normal,
    tipo='continua',
    valores_default=[0, 1],
    validar_parametros=validar_normal,
    tipos_parametros=['float', 'float']
)


# Finalmente, muestra la interfaz interactiva
va.mostrar_interactividad()

Output()

Output()